In [6]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor
import calendar
lbl = preprocessing.LabelEncoder()

In [3]:
# read all the data
air_visits = pd.read_csv('../input/air_visit_data.csv')
air_reserve = pd.read_csv('../input/air_reserve.csv')
air_store_info = pd.read_csv('../input/air_store_info.csv')
hpg_reserve = pd.read_csv('../input/hpg_reserve.csv')
hpg_store_info = pd.read_csv('../input/hpg_store_info.csv')
relation = pd.read_csv('../input/store_id_relation.csv')
hol = pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
sample_submission = pd.read_csv('../input/sample_submission.csv')
weather = pd.read_csv('../input/weather.csv')

In [7]:
hpg_reserve = pd.merge(hpg_reserve, relation, how='inner', on=['hpg_store_id'])

In [8]:
# transfer hpg_reserve to date and find the date difference
hpg_reserve['visit_datetime'] = pd.to_datetime(hpg_reserve['visit_datetime'])
hpg_reserve['visit_datetime'] = hpg_reserve['visit_datetime'].dt.date
hpg_reserve['reserve_datetime'] = pd.to_datetime(hpg_reserve['reserve_datetime'])
hpg_reserve['reserve_datetime'] = hpg_reserve['reserve_datetime'].dt.date
hpg_reserve['reserve_datetime_diff'] = hpg_reserve.apply(lambda r:(r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
tmp1 = hpg_reserve.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
tmp2 = hpg_reserve.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
hpg_reserve = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

In [9]:
# transfer air_reserve to date and find the date difference
air_reserve['visit_datetime'] = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['visit_datetime'] = air_reserve['visit_datetime'].dt.date
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])
air_reserve['reserve_datetime'] = air_reserve['reserve_datetime'].dt.date
air_reserve['reserve_datetime_diff'] = air_reserve.apply(lambda r:(r['visit_datetime'] - r['reserve_datetime']).days, axis=1)
tmp1 = air_reserve.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})
tmp2 = air_reserve.groupby(['air_store_id','visit_datetime'], as_index=False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})
air_reserve = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])

In [10]:
# transfer air_visits
air_visits['visit_date'] = pd.to_datetime(air_visits['visit_date'])
air_visits['dow'] = air_visits['visit_date'].dt.dayofweek
air_visits['visit_date'] = air_visits['visit_date'].dt.date

In [11]:
# transfer weather
weather['visit_date'] = pd.to_datetime(weather['visit_date'])
weather['visit_date'] = weather['visit_date'].dt.date

In [12]:
# transfer sample submission
sample_submission['visit_date'] = sample_submission['id'].map(lambda x:str(x).split('_')[2])
sample_submission['air_store_id'] = sample_submission['id'].map(lambda x: '_'.join(str(x).split('_')[:2]))
sample_submission['visit_date'] = pd.to_datetime(sample_submission['visit_date'])
sample_submission['dow'] = sample_submission['visit_date'].dt.dayofweek
sample_submission['visit_date'] = sample_submission['visit_date'].dt.date

In [13]:
# Get all the features of test data
unique_stores = sample_submission['air_store_id'].unique()
stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], axis=0, ignore_index=True).reset_index(drop=True)

tmp = air_visits.groupby(['air_store_id','dow'], as_index=False)['visitors'].min().rename(columns={'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 
tmp = air_visits.groupby(['air_store_id','dow'], as_index=False)['visitors'].mean().rename(columns={'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = air_visits.groupby(['air_store_id','dow'], as_index=False)['visitors'].median().rename(columns={'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = air_visits.groupby(['air_store_id','dow'], as_index=False)['visitors'].max().rename(columns={'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow'])
tmp = air_visits.groupby(['air_store_id','dow'], as_index=False)['visitors'].count().rename(columns={'visitors':'count_observations'})
stores = pd.merge(stores, tmp, how='left', on=['air_store_id','dow']) 

stores = pd.merge(stores, air_store_info, how='left', on=['air_store_id']) 

In [14]:
stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
tmp = pd.get_dummies(stores['air_genre_name'])
stores = stores.merge(tmp, how='left', left_index=True, right_index=True)
tmp = pd.get_dummies(stores['air_area_name'].map(lambda x:str(x).split(' ')[0]))
stores = stores.merge(tmp, how='left', left_index=True, right_index=True)

In [15]:
# Modify HOL
hol['visit_date'] = pd.to_datetime(hol['visit_date'])
hol['year_2016'] = hol['visit_date'].dt.year.astype(str)
hol['year_2016'] = hol['year_2016'].replace('2016', 1).replace('2017', 0)
hol['month'] = hol['visit_date'].dt.month
hol['visit_date'] = hol['visit_date'].dt.date
hol['month'] = hol['month'].apply(lambda x: calendar.month_abbr[x])
tmp = pd.get_dummies(hol['day_of_week'])
hol = hol.merge(tmp, how='left', left_index=True, right_index=True)
tmp = pd.get_dummies(hol['month'])
hol = hol.merge(tmp, how='left', left_index=True, right_index=True)

In [16]:
# merge train and test with holiday and stores
train = pd.merge(air_visits, hol, how='left', on=['visit_date']) 
test = pd.merge(sample_submission, hol, how='left', on=['visit_date'])
train = pd.merge(train, stores, how='inner', on=['air_store_id','dow']) 
test = pd.merge(test, stores, how='left', on=['air_store_id','dow'])

In [19]:
air_visits[air_visits['air_store_id'] == 'air_0328696196e46f18']

,air_store_id,visit_date,visitors,dow
245105,air_0328696196e46f18,2016-07-03,11,6
245106,air_0328696196e46f18,2016-07-04,4,0
245107,air_0328696196e46f18,2016-07-05,6,1
245108,air_0328696196e46f18,2016-07-06,4,2
245109,air_0328696196e46f18,2016-07-07,8,3
245110,air_0328696196e46f18,2016-07-08,8,4
245111,air_0328696196e46f18,2016-07-09,12,5
245112,air_0328696196e46f18,2016-07-10,4,6
245113,air_0328696196e46f18,2016-07-11,4,0
245114,air_0328696196e46f18,2016-07-12,8,1


In [18]:
train[train['air_store_id'] == 'air_0328696196e46f18']

,air_store_id,visit_date,visitors,dow,day_of_week,holiday_flg,year_2016,month,Friday,Monday,...,Yakiniku Korean food,Fukuoka-ken,Hiroshima-ken,Hokkaidō,Hyōgo-ken,Miyagi-ken,Niigata-ken,Shizuoka-ken,Tōkyō-to,Ōsaka-fu
243465,air_0328696196e46f18,2016-07-03,11,6,Sunday,0,1,Jul,0,0,...,0,0,0,0,0,0,0,0,0,1
243466,air_0328696196e46f18,2016-07-10,4,6,Sunday,0,1,Jul,0,0,...,0,0,0,0,0,0,0,0,0,1
243467,air_0328696196e46f18,2016-07-17,3,6,Sunday,0,1,Jul,0,0,...,0,0,0,0,0,0,0,0,0,1
243468,air_0328696196e46f18,2016-07-24,19,6,Sunday,0,1,Jul,0,0,...,0,0,0,0,0,0,0,0,0,1
243469,air_0328696196e46f18,2016-07-31,7,6,Sunday,0,1,Jul,0,0,...,0,0,0,0,0,0,0,0,0,1
243470,air_0328696196e46f18,2016-08-28,8,6,Sunday,0,1,Aug,0,0,...,0,0,0,0,0,0,0,0,0,1
243471,air_0328696196e46f18,2016-10-30,6,6,Sunday,0,1,Oct,0,0,...,0,0,0,0,0,0,0,0,0,1
243472,air_0328696196e46f18,2016-11-06,7,6,Sunday,0,1,Nov,0,0,...,0,0,0,0,0,0,0,0,0,1
243473,air_0328696196e46f18,2017-01-01,9,6,Sunday,1,0,Jan,0,0,...,0,0,0,0,0,0,0,0,0,1
243474,air_0328696196e46f18,2017-01-08,2,6,Sunday,0,0,Jan,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
train

,air_store_id,visit_date,visitors,dow,day_of_week,holiday_flg,year_2016,month,Friday,Monday,...,Yakiniku Korean food,Fukuoka-ken,Hiroshima-ken,Hokkaidō,Hyōgo-ken,Miyagi-ken,Niigata-ken,Shizuoka-ken,Tōkyō-to,Ōsaka-fu
0,air_ba937bf13d40fb24,2016-01-13,25,2,Wednesday,0,1,Jan,0,0,...,0,0,0,0,0,0,0,0,1,0
1,air_ba937bf13d40fb24,2016-01-20,31,2,Wednesday,0,1,Jan,0,0,...,0,0,0,0,0,0,0,0,1,0
2,air_ba937bf13d40fb24,2016-01-27,24,2,Wednesday,0,1,Jan,0,0,...,0,0,0,0,0,0,0,0,1,0
3,air_ba937bf13d40fb24,2016-02-03,18,2,Wednesday,0,1,Feb,0,0,...,0,0,0,0,0,0,0,0,1,0
4,air_ba937bf13d40fb24,2016-02-10,32,2,Wednesday,0,1,Feb,0,0,...,0,0,0,0,0,0,0,0,1,0
5,air_ba937bf13d40fb24,2016-02-17,17,2,Wednesday,0,1,Feb,0,0,...,0,0,0,0,0,0,0,0,1,0
6,air_ba937bf13d40fb24,2016-02-24,21,2,Wednesday,0,1,Feb,0,0,...,0,0,0,0,0,0,0,0,1,0
7,air_ba937bf13d40fb24,2016-03-02,21,2,Wednesday,0,1,Mar,0,0,...,0,0,0,0,0,0,0,0,1,0
8,air_ba937bf13d40fb24,2016-03-09,23,2,Wednesday,0,1,Mar,0,0,...,0,0,0,0,0,0,0,0,1,0
9,air_ba937bf13d40fb24,2016-03-16,25,2,Wednesday,0,1,Mar,0,0,...,0,0,0,0,0,0,0,0,1,0


In [27]:
type(train.iloc[0]['air_store_id'])

str

In [15]:
#merge train and test with reserve
train = pd.merge(train, air_reserve, how='left', on=['air_store_id','visit_date']) 
test = pd.merge(test, air_reserve, how='left', on=['air_store_id','visit_date'])
train = pd.merge(train, hpg_reserve, how='left', on=['air_store_id','visit_date']) 
test = pd.merge(test, hpg_reserve, how='left', on=['air_store_id','visit_date'])

In [16]:
# Add Weather Features
train = pd.merge(train, weather, how='left', on=['air_store_id','visit_date']) 
test = pd.merge(test, weather, how='left', on=['air_store_id','visit_date'])

In [17]:
# Add Features 1
train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

train['total_reserv_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserv_mean'] = (train['rv2_x'] + train['rv2_y']) / 2
train['total_reserv_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y']) / 2

test['total_reserv_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserv_mean'] = (test['rv2_x'] + test['rv2_y']) / 2
test['total_reserv_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y']) / 2

In [17]:
# Add Features 2
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']

In [18]:
# Add Features 3
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

In [19]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [30]:
# Final Training set and test set
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors', 'dow', 'day_of_week', 'year', 'month', 'air_genre_name', 'air_area_name']]
train = train.fillna(-1)
test = test.fillna(-1)

In [19]:
train

,air_store_id,visit_date,visitors,dow,day_of_week,holiday_flg,year,month,year_2016,year_2017,...,extreme_temp,rainy,windy,id,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,var_max_lat,var_max_long,lon_plus_lat
0,air_ba937bf13d40fb24,2016-01-13,25,2,Wednesday,0,year_2016,Jan,1,0,...,1,0,0,air_ba937bf13d40fb24_2016-01-13,NaN,NaN,NaN,8.362564,4.521799,175.409667
1,air_ba937bf13d40fb24,2016-01-20,31,2,Wednesday,0,year_2016,Jan,1,0,...,1,0,0,air_ba937bf13d40fb24_2016-01-20,NaN,NaN,NaN,8.362564,4.521799,175.409667
2,air_ba937bf13d40fb24,2016-01-27,24,2,Wednesday,0,year_2016,Jan,1,0,...,1,0,0,air_ba937bf13d40fb24_2016-01-27,NaN,NaN,NaN,8.362564,4.521799,175.409667
3,air_ba937bf13d40fb24,2016-02-03,18,2,Wednesday,0,year_2016,Feb,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-02-03,NaN,NaN,NaN,8.362564,4.521799,175.409667
4,air_ba937bf13d40fb24,2016-02-10,32,2,Wednesday,0,year_2016,Feb,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-02-10,NaN,NaN,NaN,8.362564,4.521799,175.409667
5,air_ba937bf13d40fb24,2016-02-17,17,2,Wednesday,0,year_2016,Feb,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-02-17,NaN,NaN,NaN,8.362564,4.521799,175.409667
6,air_ba937bf13d40fb24,2016-02-24,21,2,Wednesday,0,year_2016,Feb,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-02-24,NaN,NaN,NaN,8.362564,4.521799,175.409667
7,air_ba937bf13d40fb24,2016-03-02,21,2,Wednesday,0,year_2016,Mar,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-03-02,NaN,NaN,NaN,8.362564,4.521799,175.409667
8,air_ba937bf13d40fb24,2016-03-09,23,2,Wednesday,0,year_2016,Mar,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-03-09,NaN,NaN,NaN,8.362564,4.521799,175.409667
9,air_ba937bf13d40fb24,2016-03-16,25,2,Wednesday,0,year_2016,Mar,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-03-16,NaN,NaN,NaN,8.362564,4.521799,175.409667


In [25]:
from sklearn.model_selection import train_test_split
X=train
y=list(train['visitors'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, , random_state=42)

In [38]:
test.to_csv("test_data.csv", index=False)

In [22]:
train

,air_store_id,visit_date,visitors,dow,day_of_week,holiday_flg,year,month,year_2016,year_2017,...,extreme_temp,rainy,windy,id,total_reserv_sum,total_reserv_mean,total_reserv_dt_diff_mean,var_max_lat,var_max_long,lon_plus_lat
0,air_ba937bf13d40fb24,2016-01-13,25,2,Wednesday,0,year_2016,Jan,1,0,...,1,0,0,air_ba937bf13d40fb24_2016-01-13,-1.0,-1.0,-1.0,8.362564,4.521799,175.409667
1,air_ba937bf13d40fb24,2016-01-20,31,2,Wednesday,0,year_2016,Jan,1,0,...,1,0,0,air_ba937bf13d40fb24_2016-01-20,-1.0,-1.0,-1.0,8.362564,4.521799,175.409667
2,air_ba937bf13d40fb24,2016-01-27,24,2,Wednesday,0,year_2016,Jan,1,0,...,1,0,0,air_ba937bf13d40fb24_2016-01-27,-1.0,-1.0,-1.0,8.362564,4.521799,175.409667
3,air_ba937bf13d40fb24,2016-02-03,18,2,Wednesday,0,year_2016,Feb,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-02-03,-1.0,-1.0,-1.0,8.362564,4.521799,175.409667
4,air_ba937bf13d40fb24,2016-02-10,32,2,Wednesday,0,year_2016,Feb,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-02-10,-1.0,-1.0,-1.0,8.362564,4.521799,175.409667
5,air_ba937bf13d40fb24,2016-02-17,17,2,Wednesday,0,year_2016,Feb,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-02-17,-1.0,-1.0,-1.0,8.362564,4.521799,175.409667
6,air_ba937bf13d40fb24,2016-02-24,21,2,Wednesday,0,year_2016,Feb,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-02-24,-1.0,-1.0,-1.0,8.362564,4.521799,175.409667
7,air_ba937bf13d40fb24,2016-03-02,21,2,Wednesday,0,year_2016,Mar,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-03-02,-1.0,-1.0,-1.0,8.362564,4.521799,175.409667
8,air_ba937bf13d40fb24,2016-03-09,23,2,Wednesday,0,year_2016,Mar,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-03-09,-1.0,-1.0,-1.0,8.362564,4.521799,175.409667
9,air_ba937bf13d40fb24,2016-03-16,25,2,Wednesday,0,year_2016,Mar,1,0,...,0,0,0,air_ba937bf13d40fb24_2016-03-16,-1.0,-1.0,-1.0,8.362564,4.521799,175.409667


In [68]:
# Get X_train and Y_train
X = train.copy()
X_test = test[nn_col].copy()

value_scaler = preprocessing.MinMaxScaler()
for vcol in value_col:
    X[vcol] = value_scaler.fit_transform(X[vcol].values.astype(np.float64).reshape(-1, 1))
    X_test[vcol] = value_scaler.transform(X_test[vcol].values.astype(np.float64).reshape(-1, 1))

X_train = list(X[nn_col].T.as_matrix())
Y_train = np.log1p(X['visitors']).values
nn_train = [X_train, Y_train]
nn_test = [list(X_test[nn_col].T.as_matrix())]

In [29]:
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.15, random_state=3,
                    n_estimators=200, subsample=0.8, max_depth =10)
# model2 = neighbors.KNeighborsRegressor(n_jobs=-1, n_neighbors=4)
model2 = ensemble.RandomForestRegressor(n_estimators=13, random_state=3, max_depth=18,
                                        min_weight_fraction_leaf=0.0002)
model3 = XGBRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, 
                      colsample_bytree=0.8, max_depth =10)

In [31]:
model1.fit(train[col], np.log1p(train['visitors'].values))

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.2, loss='ls', max_depth=10, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=3, subsample=0.8, verbose=0,
             warm_start=False)

In [32]:
preds1 = model1.predict(train[col])

In [33]:
print('RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds1))

RMSE GradientBoostingRegressor:  0.353490873343


In [34]:
preds1 = model1.predict(test[col])

In [35]:
test['visitors'] = preds1
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)

In [36]:
result = test[['id', 'visitors']]

In [37]:
result.to_csv('result1.csv',index=False)